In [91]:
# Import
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
from skimage.feature import hog
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.svm import LinearSVC
import time

In [80]:
def plot_2(img1,img2):
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 9))
    f.tight_layout()
    ax1.imshow(img1)
    ax2.imshow(img2)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

## Load Images

In [81]:
cars = []
notcars = []
images = glob.glob('vehicles/vehicles/*/*.png')
for image in images:
    cars.append(image)
    
images = glob.glob('non-vehicles/non-vehicles/*/*.png')
for image in images:
    notcars.append(image)

    
#images2 = glob.glob('vehicles/vehicles/KITTI_*/*.png')
#cars.append(images)
print("cars ",len(cars))
print("non_cars ", len(notcars))
image = np.uint8(cv2.imread(cars[0]))
print("MAx ",np.max(image))
print("MIN",np.min(image))
image2 = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

#plot_2(image, image2)


cars  8792
non_cars  8968
MAx  192
MIN 33


## Feature Extraction

In [82]:
# Define a function to compute binned color features  
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
    return features

# Define a function to compute color histogram features  
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

# Define a function to return HOG features and visualization
# Features will always be the first element of the return
# Image data will be returned as the second element if visualize= True
# Otherwise there is no second return element

def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=True, 
                     feature_vec=True):
                         
    # TODO: Complete the function body and returns
    
    #pass
    feature_vector = hog(img, orientations=orient,
                          pixels_per_cell=(pix_per_cell, pix_per_cell), 
                          cells_per_block=(cell_per_block, cell_per_block), 
                          visualise=vis, feature_vector=feature_vec,
                          block_norm="L2-Hys")
    return hog_features

def hog_features(feature_image,orient,pix_per_cell,cell_per_block,vis=True,feature_vec=True,hog_channel):
    # Call get_hog_features() with vis=False, feature_vec=True
    if hog_channel == 'ALL':
        hog_features = []
        for channel in range(feature_image.shape[2]):
            hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                orient, pix_per_cell, cell_per_block, 
                                vis=False, feature_vec=True))
        hog_features = np.ravel(hog_features)        
    else:
        hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                    pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        
    return hog_features

In [96]:
# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(imgs, cspace='RGB', spatial_size=(32, 32),
                        hist_bins=32, hist_range=(0, 256), hog_channel = 'ALL'):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
        # Read in each one by one
        # apply color conversion if other than 'RGB'
        # Apply bin_spatial() to get spatial color features
        # Apply color_hist() to get color histogram features
        # Append the new feature vector to the features list
    # Return list of feature vectors
    orient =
    pix_per_cell=
    cell_per_block=
    
    for img in imgs:
        image = cv2.imread(img)
        if cspace == 'BGR':
            feature_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        elif cspace == 'HLS':
            feature_image = cv2.cvtColor(image, cv2.COLOR_HLS2RGB)
        elif cspace == 'HSV':
            feature_image = cv2.cvtColor(image, cv2.COLOR_HSV2RGB)
        elif cspace == 'YUV':
            feature_image = cv2.cvtColor(image, cv2.COLOR_YUV2RGB)
        else:
            feature_image = np.copy(image)
        
        spatial_feature = bin_spatial(feature_image, size = spatial_size)
        color_feature = color_hist(feature_image, nbins = hist_bins, bins_range = hist_range)
        hog_features = 
        
       
        
        features.append(np.concatenate((spatial_feature,color_feature,hog_features)))
    return features

In [97]:
car_features = extract_features(cars, cspace='BGR', spatial_size=(16, 16),
                        hist_bins=32, hist_range=(0, 256),hog_channel = 'ALL')
notcar_features = extract_features(notcars, cspace='BGR', spatial_size=(16, 16),
                        hist_bins=32, hist_range=(0, 256),hog_channel = 'ALL')

if len(car_features) > 0:
    # Create an array stack of feature vectors
    X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
    # Fit a per-column scaler
    X_scaler = StandardScaler().fit(X)
    # Apply the scaler to X
    scaled_X = X_scaler.transform(X)
    car_ind = np.random.randint(0, len(cars))
    # Plot an example of raw and scaled features
    fig = plt.figure(figsize=(12,4))
    plt.subplot(131)
    plt.imshow(mpimg.imread(cars[car_ind]))
    plt.title('Original Image')
    plt.subplot(132)
    plt.plot(X[car_ind])
    plt.title('Raw Features')
    plt.subplot(133)
    plt.plot(scaled_X[car_ind])
    plt.title('Normalized Features')
    fig.tight_layout()
else: 
    print('Your function only returns empty feature vectors...')

NameError: name 'orient' is not defined

## Split into Training and validation

In [93]:
# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float64)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=rand_state)
    
# Fit a per-column scaler only on the training data
X_scaler = StandardScaler().fit(X_train)
# Apply the scaler to X_train and X_test
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

#print('Using spatial binning of:',spatial,'and', histbin,'histogram bins')
print('Feature vector length:', len(X_train[0]))


Feature vector length: 864


## Linear SVM

In [94]:
# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()
n_predict = 10
print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')

9.3 Seconds to train SVC...
Test Accuracy of SVC =  0.9617
My SVC predicts:  [1. 0. 0. 0. 0. 0. 1. 0. 1. 1.]
For these 10 labels:  [1. 0. 0. 0. 0. 0. 1. 0. 1. 1.]
0.0 Seconds to predict 10 labels with SVC


In [95]:
print(len(X_train))

14208


## Sliding Window

## PIPELINE

In [ ]:
def pipeline(image):
    